# YouTube comments

Here I used google api. There may be other tools. 


* https://developers.google.com/youtube/v3/docs/commentThreads/list
* https://developers.google.com/youtube/v3/docs/comments/list

Check out the link "See code samples"


Get authorization: I used API key
https://developers.google.com/youtube/registering_an_application


Output: also CSV. Ideally in the same/compatible column schema with NYTimes results (e.g. common fields like id, text, timestamp). 

In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from typing import Dict

The following functions could be organized as a python class. 

It looks like YouTube API lets us get at the nested structure of comments, which the NYTimes API lacks. We need a way to save the nested structure (e.g. add a field parent_id)

In [4]:
def get_comments(video_id: str) -> Dict:
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = os.environ.get("GOOGLE_API_KEY")

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="snippet",
        order="time",
        videoId=video_id,
        maxResults = 10,
    )
    response = request.execute()
    return response # fields ['kind', 'etag', 'nextPageToken', 'pageInfo', 'items']

def get_child_comments(Id: str) -> Dict:
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = os.environ.get("GOOGLE_API_KEY")

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.comments().list(
        part="snippet",
        parentId=Id,
        maxResults = 50
    )
    response = request.execute()
    return response # four fields

In [5]:
video_id = "Rt_UqUm38BI" # https://www.youtube.com/watch?v=Rt_UqUm38BI
res = get_comments(video_id)
for comment in res["items"]:
    text = comment["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
    children = get_child_comments(comment["id"])["items"]
    print(comment["id"])
    print(text)
    print(f"{len(children)} child comments")
    print()

UgyS2G3LIasPvwj-YUR4AaABAg
If this movie does not do a billion dollar then there is something wrong with this reality.
0 child comments

Ugwzhp0v37MWTrD0s1V4AaABAg
Every Marvel movie has a dialogue which goes like "the greatest threat to our universe..."
0 child comments

UgwF1lU05mZAKk9QtKt4AaABAg
I'm so happy that they didn't release this on the internet right away, for seeing it in cinema for the first time was incredible
0 child comments

UgxcTI4jyCuESWhocON4AaABAg
I am so excited
0 child comments

Ugya8Vchj4chqZzgZyV4AaABAg
Maybe our Dr strange and evil strange team up against a greater threat 🤷
That would be cool imo
0 child comments

UgxnlXRnxw6jItmf22N4AaABAg
The simple fact that Sam raimi is directing the next dr strange is why I'm super excited for it.  Can't wait to see what horror elements he slips in because we all know Sam raimi can't help himself
1 child comments

UgyGwAem-xPMmCvfy014AaABAg
Sam Raimi directed this. My hopes are up for a Marvel movie without a ton of sill